<a href="https://colab.research.google.com/github/FranMoretti/FranMoretti/blob/main/Proveedores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Primero, asegúrate de tener las bibliotecas necesarias
import pandas as pd
import numpy as np

# Carga el archivo Excel
file_path = '30500010912 - BNA.xlsx'  # Ajusta la ruta según tu archivo
df = pd.read_excel(file_path)

In [ ]:
# 1) Top 10 de mayores montos por 'CPR CUIT VDOR APRO' según la columna 'CPR TOT OPE'
top_10_montos = df.groupby('CPR CUIT VDOR APRO').agg({'CPR TOT OPE': 'sum', 'CPR NOMBRE VDOR': 'first'}).reset_index()
top_10_montos = top_10_montos.sort_values(by='CPR TOT OPE', ascending=False).head(10)
top_10_montos['Cantidad de registros'] = df.groupby('CPR CUIT VDOR APRO').size().reset_index(name='Cantidad')['Cantidad']
top_10_montos['Monto $'] = top_10_montos['CPR TOT OPE']
top_10_montos = top_10_montos[['CPR CUIT VDOR APRO', 'CPR NOMBRE VDOR', 'Cantidad de registros', 'Monto $']]
top_10_montos.index = range(1, len(top_10_montos) + 1)
top_10_montos['Monto $'] = top_10_montos['Monto $'].apply(lambda x: f'${x:,.2f}')

# 2) Top 10 de mayores cantidades de registros, excluyendo los 'CPR CUIT VDOR APRO' ya en top_10_montos
top_10_cantidades = df.groupby('CPR CUIT VDOR APRO').size().reset_index(name='Cantidad')
top_10_cantidades = top_10_cantidades[~top_10_cantidades['CPR CUIT VDOR APRO'].isin(top_10_montos['CPR CUIT VDOR APRO'])]
top_10_cantidades = top_10_cantidades.sort_values(by='Cantidad', ascending=False).head(10)
top_10_cantidades['Monto $'] = df.groupby('CPR CUIT VDOR APRO')['CPR TOT OPE'].sum().reset_index(name='CPR TOT OPE')['CPR TOT OPE']
top_10_cantidades['CPR NOMBRE VDOR'] = df.groupby('CPR CUIT VDOR APRO')['CPR NOMBRE VDOR'].first().reset_index(name='CPR NOMBRE VDOR')['CPR NOMBRE VDOR']
top_10_cantidades = top_10_cantidades[['CPR CUIT VDOR APRO', 'CPR NOMBRE VDOR', 'Cantidad', 'Monto $']]
top_10_cantidades.index = range(1, len(top_10_cantidades) + 1)
top_10_cantidades['Monto $'] = top_10_cantidades['Monto $'].apply(lambda x: f'${x:,.2f}')

# 3) Top 5 de clientes más habituales, con registros en 9 o 10 períodos, ordenado por monto
# Agrupación por cliente y período
clientes_habituales = df.groupby(['CPR CUIT VDOR APRO', 'PERIODO'])['CPR TOT OPE'].sum().reset_index()
# Contar períodos únicos por cliente
clientes_habituales_count = clientes_habituales.groupby('CPR CUIT VDOR APRO')['PERIODO'].nunique().reset_index()
clientes_habituales_count.columns = ['CPR CUIT VDOR APRO', 'Periodos unicos']
# Filtrar clientes con 9 o más períodos únicos
clientes_habituales_count = clientes_habituales_count[clientes_habituales_count['Periodos unicos'] >= 9]
# Calcular el monto total por cliente
clientes_habituales_monto = df.groupby('CPR CUIT VDOR APRO')['CPR TOT OPE'].sum().reset_index()
clientes_habituales_monto.columns = ['CPR CUIT VDOR APRO', 'Monto $']
# Unir con los nombres y contar registros
clientes_habituales = clientes_habituales_count.merge(clientes_habituales_monto, on='CPR CUIT VDOR APRO')
# Crear diccionario único para 'CPR NOMBRE VDOR'
nombre_dict = df[['CPR CUIT VDOR APRO', 'CPR NOMBRE VDOR']].drop_duplicates().set_index('CPR CUIT VDOR APRO')['CPR NOMBRE VDOR'].to_dict()
# Mapear los nombres usando el diccionario
clientes_habituales['CPR NOMBRE VDOR'] = clientes_habituales['CPR CUIT VDOR APRO'].map(nombre_dict)
# Calcular cantidad de registros por cliente
clientes_habituales['Cantidad de registros'] = df.groupby('CPR CUIT VDOR APRO').size().reindex(clientes_habituales['CPR CUIT VDOR APRO']).values
# Excluir clientes ya presentes en top 10 montos y top 10 cantidades
excluir = set(top_10_montos['CPR CUIT VDOR APRO']).union(set(top_10_cantidades['CPR CUIT VDOR APRO']))
clientes_habituales = clientes_habituales[~clientes_habituales['CPR CUIT VDOR APRO'].isin(excluir)]
# Seleccionar el top 5 por monto
clientes_habituales = clientes_habituales.sort_values(by='Monto $', ascending=False).head(5)
# Reordenar y aplicar formato
clientes_habituales = clientes_habituales[['CPR CUIT VDOR APRO', 'CPR NOMBRE VDOR', 'Cantidad de registros', 'Monto $']]
clientes_habituales.index = range(1, len(clientes_habituales) + 1)
clientes_habituales['Monto $'] = clientes_habituales['Monto $'].apply(lambda x: f'${x:,.2f}')


# 4) Top 5 de clientes menos habituales, con registros en solo 1 o 2 períodos, ordenado por monto
# Agrupación por cliente y período
clientes_no_habituales = df.groupby(['CPR CUIT VDOR APRO', 'PERIODO'])['CPR TOT OPE'].sum().reset_index()
# Contar períodos únicos por cliente
clientes_no_habituales_count = clientes_no_habituales.groupby('CPR CUIT VDOR APRO')['PERIODO'].nunique().reset_index()
clientes_no_habituales_count.columns = ['CPR CUIT VDOR APRO', 'Periodos unicos']
# Filtrar clientes con 1 o 2 períodos únicos
clientes_no_habituales_count = clientes_no_habituales_count[clientes_no_habituales_count['Periodos unicos'] <= 2]
# Calcular el monto total por cliente
clientes_no_habituales_monto = df.groupby('CPR CUIT VDOR APRO')['CPR TOT OPE'].sum().reset_index()
clientes_no_habituales_monto.columns = ['CPR CUIT VDOR APRO', 'Monto $']
# Unir con los nombres y contar registros
clientes_no_habituales = clientes_no_habituales_count.merge(clientes_no_habituales_monto, on='CPR CUIT VDOR APRO')
# Crear diccionario único para 'CPR NOMBRE VDOR'
nombre_dict = df[['CPR CUIT VDOR APRO', 'CPR NOMBRE VDOR']].drop_duplicates().set_index('CPR CUIT VDOR APRO')['CPR NOMBRE VDOR'].to_dict()
# Mapear los nombres usando el diccionario
clientes_no_habituales['CPR NOMBRE VDOR'] = clientes_no_habituales['CPR CUIT VDOR APRO'].map(nombre_dict)
# Calcular cantidad de registros por cliente
clientes_no_habituales['Cantidad de registros'] = df.groupby('CPR CUIT VDOR APRO').size().reindex(clientes_no_habituales['CPR CUIT VDOR APRO']).values
# Excluir clientes ya presentes en top 10 montos, top 10 cantidades y top 5 habituales
excluir = set(top_10_montos['CPR CUIT VDOR APRO']).union(set(top_10_cantidades['CPR CUIT VDOR APRO'])).union(set(clientes_habituales['CPR CUIT VDOR APRO']))
clientes_no_habituales = clientes_no_habituales[~clientes_no_habituales['CPR CUIT VDOR APRO'].isin(excluir)]
# Seleccionar el top 5 por monto
clientes_no_habituales = clientes_no_habituales.sort_values(by='Monto $', ascending=False).head(5)
# Reordenar y aplicar formato
clientes_no_habituales = clientes_no_habituales[['CPR CUIT VDOR APRO', 'CPR NOMBRE VDOR', 'Cantidad de registros', 'Monto $']]
clientes_no_habituales.index = range(1, len(clientes_no_habituales) + 1)
clientes_no_habituales['Monto $'] = clientes_no_habituales['Monto $'].apply(lambda x: f'${x:,.2f}')

In [ ]:
top_10_montos

In [ ]:
top_10_cantidades

In [ ]:
clientes_habituales

In [ ]:
clientes_no_habituales